<a href="https://colab.research.google.com/github/ppandiri/Spotify_Playlist_Generator/blob/main/TestOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
import pandas as pd
import numpy as np

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install spotipy
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
# Set up Spotipy client
client_credentials_manager = SpotifyClientCredentials(client_id='79c44db50adf46d895fb50cdccd7a563', client_secret='7bdf4da181894bf0a96d9b2ab79bffde')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [10]:
#run SpotifyKaggle to get dataset
%run /content/drive/MyDrive/SpotifyProject/SpotifyKaggle.ipynb
df = newData

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: bhaviksinghal
Your Kaggle Key: ··········


100%|██████████| 3.01M/3.01M [00:01<00:00, 3.13MB/s]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Stored 'newData' (DataFrame)


In [25]:
# Plotting the graph (if we use three features)
"""
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df['key'], df['valence'], df['energy'], c='b', marker='o')

ax.set_xlabel('key')
ax.set_ylabel('valence')
ax.set_zlabel('energy')

plt.show()

"""

# Features collected from Population Correlation
column_indices = [df.columns.get_loc('key'), df.columns.get_loc('valence'),df.columns.get_loc('energy'), df.columns.get_loc('speechiness')]

#Getting a suggestion from user input
while True:
  try:
    user_song = input("Give me a song to generate a playlist from: ")
    user_artist = input("Give me the artist who made that song: ")
    query = "track:{} artist:{}".format(user_song, user_artist)
    results = sp.search(q=query, type='track')
    track_id = results['tracks']['items'][0]['id']
    print(track_id)
    print("Track ID for '{}' by '{}' is: {}".format(user_song, user_artist, track_id))
    break
  except:
    print("This song is not in the Spotify database")
audio_features = sp.audio_features(track_id)
track_features = audio_features[0]  # audio_features is a list, so we take the first (and only) item
features_series = pd.Series({
    'key': track_features['key'],
    'valence': track_features['valence'],
    'energy': track_features['energy'],
    'speechiness': track_features['speechiness']
})
print("Features Series:")
print(features_series)
chosen_song = features_series
# Calculate Euclidean distances (added speechiness so voice will also have a factor (i.e. instrumentals submitted will return instrumentals))
distances = np.apply_along_axis(lambda x: euclidean(chosen_song, x), 1, df[['key', 'valence', 'energy', 'speechiness']].values)

# Find the indices of the 20 songs with the smallest distances
close_song_list = []
close_artist_list = []
i = 0
while len(close_song_list) < 20:
    closest_songs_index = np.argsort(distances)[i]
    # This prevents the same song from being in the playlist twice
    if (df.iloc[closest_songs_index]["track_name"] != df.at[row_index,"track_name"]) and (df.iloc[closest_songs_index]["track_name"] not in close_song_list):
        close_song_list.append(df.iloc[closest_songs_index]["track_name"])
        close_artist_list.append(df.iloc[closest_songs_index]["track_artist"])

    i += 1



#closest_songs = df.iloc[closest_songs_indices]["track_name"].tolist()
#closest_artist = df.iloc[closest_songs_indices]["track_artist"].tolist()

# Display the closest songs
print("Closest Songs:")
print(close_song_list)
print(close_artist_list)
#print(closest_artist)

Give me a song to generate a playlist from: See You Again
Give me the artist who made that song: Tyler the Creator
7KA4W4McWYRpgf0fWsJZWB
Track ID for 'See You Again' by 'Tyler the Creator' is: 7KA4W4McWYRpgf0fWsJZWB
Features Series:
key            6.0000
valence        0.6200
energy         0.5590
speechiness    0.0959
dtype: float64
Closest Songs:
['See You Again (feat. Kali Uchis)', "God's Great Dance Floor - Remix", 'God Is Real', 'Dibiase', 'Promise Me', 'Feelings - KAJ Remix', "I'm So Glad", 'Road Trips', 'hot girl bummer (with Khea) - Khea Remix', "Tonight's The Night", 'Sharpness', 'Quiero - Reggaeton Romantico', 'Hola Beba', 'Louquinho', 'No Goodbye', 'Love Thang', 'Nadie Sabe', 'hot girl bummer', 'ANOTHERLOVE', 'Send My Love (To Your New Lover)']
['Tyler, The Creator', 'Reyer', 'India.Arie', '03 Greedo', 'Ballin Entertainment', 'Deep Chills', 'Jamie Jones', 'Kyle Watson', 'blackbear', 'Blackstreet', 'Jamie Woon', 'Cheeky El Exotico', 'Farruko', 'Jão', 'Paul Kalkbrenner', 'Mic